In [1]:
from torchvision import utils
from basic_fcn import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time

In [2]:
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=10,
                          num_workers=4,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=2,
                          num_workers=4,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=10,
                          num_workers=4,
                          shuffle=True)

In [3]:
#dataiter = iter(train_loader)
#images, unoHot, labels = dataiter.next()

In [4]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.xavier_uniform_(m.bias.data.unsqueeze(0)) #add unsqueeze here 
        
epochs     = 100
criterion = nn.CrossEntropyLoss()
# Choose an appropriate loss function from https://pytorch.org/docs/stable/_modules/torch/nn/modules/loss.html
fcn_model = FCN(n_class=n_class)
fcn_model.apply(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-3)

In [8]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    fcn_model = fcn_model.cuda()
    
def train():
    for epoch in range(epochs):
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda() # Move your inputs onto the gpu
                labels = tar.cuda() # Move your labels onto the gpu
            else:
                inputs, labels = X, tar # Unpack variables into inputs and labels

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        #torch.save(fcn_model, 'best_model')

        #val(epoch)
        fcn_model.train()


def val(epoch):
    fcn_model.eval()
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    valLoss = 0
    valAcc = 0
    numIter = 0
    
    soMuchTension = torch.Tensor([0 for i in range(n_class)]) #intersection
    theresATigerInTheBathroom = torch.Tensor([0 for i in range(n_class)]) #union
    
    for iter, (X, tar, Y) in enumerate(val_loader):
        numIter +=1
        if use_gpu:
            inputs = X.cuda() # Move your inputs onto the gpu
            labels = Y.cuda()
        else:
            inputs, oneHotLabels, labels = X, tar, Y # Unpack variables into inputs and labels
        
        outputs = fcn_model(inputs)   
        valLoss += criterion(outputs, labels).item()
            
        #if use_gpu: #manage memory
        #    del labels
        #    torch.cuda.empty_cache() 
        #    oneHotLabels = tar.cuda() # Move your labels onto the gpu
            
        speakSoftlyAndCarryABigStick = F.softmax(outputs, dim = 1) #softmax along the number of class dimension
        indexes = torch.argmax(speakSoftlyAndCarryABigStick, dim = 1) #get the argmax along the channel dimension
            
        #goGoGadgetSkis = torch.zeros_like(oneHotLabels).scatter(1,indexes.unsqueeze(1), 1.0) #transform into one-hot encoding
        
        #print("Unique predictions...")
        #print(torch.unique(indexes))
        #print(indexes.size())
        #print("unique labels")
        #print(torch.unique(labels))
        #print(labels.size())
        #print(indexes)
        #print(labels)
        
        valAcc += pixel_acc(indexes, labels) 
        
        theEyesHaveIt, spaceBalls = iou(indexes, labels, n_class) #intersection, union
        
        soMuchTension = soMuchTension + theEyesHaveIt
        theresATigerInTheBathroom = theresATigerInTheBathroom + spaceBalls
      
        
        print(numIter)
            
        #manage memory 
        
        if use_gpu:
            del inputs, labels, outputs, indexes, speakSoftlyAndCarryABigStick, theEyesHaveIt, spaceBalls
            torch.cuda.empty_cache() 
        
    
            
    valAcc = valAcc/numIter
    valLoss = valLoss/numIter
    iOweYouMoney = torch.div(soMuchTension, theresATigerInTheBathroom)

    print("The validation loss is: " + str(valLoss))
    print("The pixel accuracy is: " + str(valAcc))
    
    #print("The intersection is: " )
    #print(soMuchTension)
    #print("The union is: " )
    #print(theresATigerInTheBathroom)
    
    print("The iou is...")
    print(iOweYouMoney)
    print("The building IOU is: " + str(iOweYouMoney[11].item()))
    print("The traffic sign IOU is: " + str(iOweYouMoney[20].item()))
    print("The person IOU is: " + str(iOweYouMoney[24].item()))
    print("The car IOU is: " + str(iOweYouMoney[26].item()))
    print("The bicycle IOU is: " + str(iOweYouMoney[33].item()))
    print("The average IOU is: " + str(MeanIOU(iOweYouMoney)))
    
def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    speakSoftlyAndCarryABigStick = F.softmax(outputs, dim = 1)
    
if __name__ == "__main__":
    val(0)  # show the accuracy before training
    #train()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
The validation loss is: 3.497163146018982
The pixel accuracy is: 3.0853135108947756
The iou is...
tensor([  